# Spin decomposition of operator 32 (part 1): Spin

Spin decomposition of operator
$$
\mathcal{O}^{(32)} =  \vec k \cdot \sigma_{DM} \vec k \cdot \vec \sigma_{2}
$$
where 
$\sigma_{DM}$ is a Dark Matter spin operator, $\sigma_{2}$ is a spin operator acting on the second nucleon and $\vec k = \vec p_i - \vec p_o + \vec q/2$.

The notebook does two things:

1. It computes the intermediate results Sachin has provided

\begin{multline}
    \mathcal{O}_{(s_o s_i) m_{\sigma} m_{DM o} m_{DM i}}(x_i, x_o, \phi, \vec q)
    =
    \frac{\left(2 \sigma + 1\right)}{2 s_{o} + 1}
    \sum_{m_{s_i} m_{s_o}}
    \left\langle s_{i} m_{s_i} , \sigma m_{\sigma} \big\vert s_{o} m_{s_o} \right\rangle
    \\ \times
    \int_{0}^{2\pi} d \Phi
    \left\langle
        s_o m_{s_o}
        \big\vert
            \mathcal{O}_{DM}(x_i, x_o, \phi, \Phi, \vec q)
        \big\vert
        s_i m_{s_i}
    \right\rangle
    e^{i \left(\Phi - \frac{\phi}{2}\right) \left(m_{\sigma} - m_{DM i} + m_{DM o}\right)}
\end{multline}

2. It exports the operator used for the angular integrations which slightly differes from this expression as it keeps $m$ quantum numbers uncontracted

$$
    \mathcal{O}_{s_o m_{s_o} s_i m_{s_i} m_\lambda}(x_i, x_o, \phi, \vec q)
    =
    \int_{0}^{2\pi} d \Phi \exp( - i m_\lambda \Phi)
    \left\langle
        s_o m_{s_o}
        \big\vert
            \mathcal{O}_{DM}(x_i, x_o, \phi, \Phi, \vec q)
        \big\vert
        s_i m_{s_i}
    \right\rangle
$$

In both expressions, $x_i = cos(\theta_i)$ and $\phi_i$ are the polar and azimuthal angle of momentum $p_i$ (and similarly for $p_o$) and 
$$
    \Phi = \frac{\phi_i + \phi_o}{2}\, , \qquad
    \phi = \phi_i - \phi_o
$$

## Import modules

In [ ]:
from itertools import product

from pandas import DataFrame, Series, set_option, read_csv
from pandas.testing import assert_frame_equal

from sympy import S, Symbol, expand_trig, Function
from sympy import exquo, ExactQuotientFailed, ComputationFailed
from sympy import latex

from numpwd.integrate.analytic import SPHERICAL_BASE_SUBS, ANGLE_BASE_SUBS, integrate
from numpwd.qchannels.cg import get_cg
from numpwd.qchannels.spin import (
    expression_to_matrix,
    pauli_contract_subsystem,
    dict_to_data,
)

set_option("max_colwidth", None)

## Define the operator in a spin basis

This notebook makes uses of Sympy (Symbolic Python) which is Pythons equivalent of Mathematica. Sympy expressions are created with the `S` function and can be conveted to latex using `latex`.

At first, we construct the nucleon part of the operator. We use the notation `sigma{n}{a}` where `n` indicates which nucleon `sigma` acts on and `a` which pauli matrix the operator corresponds to.

In [ ]:
expr = S("sigma10 * (sigma21 * k1 + sigma22 * k2 + sigma23 * k3)")
expr

We need to specify $\sigma_{10} = \mathbb{1}_1$ to indicate that the spin of the first nucleon does not change.

In [ ]:
help(expression_to_matrix)

In [ ]:
mat = expression_to_matrix(expr, pauli_symbol="sigma")
mat

From this, we can also read off the DM matrix element (components where `ms1_out == ms1_in == 1/2`)

In [ ]:
up = S("1/2")
dn = -up
dm_mat = [
    {"ms_o": up, "ms_i": up, "val": S("k3")},
    {"ms_o": dn, "ms_i": dn, "val": -S("k3")},
    {"ms_o": up, "ms_i": dn, "val": S("k1 - I*k2")},
    {"ms_o": dn, "ms_i": up, "val": S("k1 + I*k2")},
]
dm_df = DataFrame(dm_mat).set_index(["ms_o", "ms_i"]).sort_index()
dm_df

Next we can contract the nucleon channels to a spin-0 or spin-1 system

In [ ]:
help(pauli_contract_subsystem)

In [ ]:
mat12 = pauli_contract_subsystem(mat)
cols = ["s_o", "ms_o", "s_i", "ms_i"]
nuc_df = DataFrame(dict_to_data(mat12, columns=cols))
nuc_df = nuc_df.set_index(cols).sort_index()
nuc_df

And finally, we can combine all nucleon channels with all DM channels (note that a few of them will be zero later on)

In [ ]:
def df_outer_product(df1, df2, suffixes=None, reset_index=False):
    tmp1 = df1.reset_index() if reset_index else df1.copy()
    tmp2 = df2.reset_index() if reset_index else df2.copy()

    if suffixes is not None:
        tmp1 = tmp1.rename(columns={key: f"{key}{suffixes[0]}" for key in tmp1.columns})
        tmp2 = tmp2.rename(columns={key: f"{key}{suffixes[1]}" for key in tmp2.columns})

    data = []
    for row1, row2 in product(tmp1.to_dict("records"), tmp2.to_dict("records")):
        data.append({**row1, **row2})

    return DataFrame(data)

In [ ]:
df = df_outer_product(nuc_df, dm_df, suffixes=["_nuc", "_dm"], reset_index=True)
df["val"] = df["val_nuc"] * df["val_dm"]
spin_df = df.set_index(
    ["ms_o_dm", "ms_i_dm", "s_o_nuc", "ms_o_nuc", "s_i_nuc", "ms_i_nuc"]
).sort_index()[["val"]]
print("Number of channels:", spin_df.size)
spin_df["val"] == spin_df["val"].apply(lambda expr: expr.expand().simplify())
spin_df.reset_index().head()

## Compare to Sachin's results

When combining expressions, they will share the factor

In [ ]:
CG = Function("CG")
pwd_fact = CG("s_i_nuc", "ms_i_nuc", "sigma", "m_sigma", "s_o_nuc", "ms_o_nuc")
pwd_fact *= S("exp(I*(m_sigma + ms_o_dm - ms_i_dm)*(Phi-phi/2))")
pwd_fact *= (2 * S("sigma") + 1) / (2 * S("s_o_nuc") + 1)
pwd_fact

The momenta are defined as

In [ ]:
momentum_subs = {f"k{n}": f"q_{n}/2 + p_i{n} - p_o{n}" for n in [1, 2, 3]}
momentum_subs

Where we eventually use that $\vec q = q\vec e_z$

In [ ]:
qz_subs = {"q_1": 0, "q_2": 0}
qz_subs

The angular substitutions are

In [ ]:
SPHERICAL_BASE_SUBS

In [ ]:
ANGLE_BASE_SUBS

Which is combined in the below function

In [ ]:
def subs_all(expr):
    return (
        expr.subs(momentum_subs)
        .subs(SPHERICAL_BASE_SUBS)
        .subs(ANGLE_BASE_SUBS)
        .subs(qz_subs)
        .rewrite("exp")
        .expand()
    )

The below code is a little bit tricky:
```
df.groupby(keys).agg(method)
```
Corresponds to a for loop structure which collects entries which share the keys and aggreates them using method.
For example, grouping by
```
["ms_o_dm", "ms_i_dm", "s_o_nuc", "s_i_nuc"]
```
would sum over all `ms_o_nuc`  and `ms_i_nuc`  contributions which have the same other keys.
The below code does exactly this sum, multiplying by the above factor and integrating over $\Phi$.

In [ ]:
def op_rank_project(tmp):
    data = dict()
    # iterate over different ms nuc values
    for row in tmp.to_dict("records"):

        # run loop over allowed sigma values
        sig_min = abs(row["s_o_nuc"] - row["s_i_nuc"])
        sig_max = abs(row["s_o_nuc"] + row["s_i_nuc"])
        for sigma in range(sig_min, sig_max + 1):
            m_sigma = row["ms_o_nuc"] - row["ms_i_nuc"]
            if abs(m_sigma) > sigma:
                continue

            # store results for unique sigma and m_sigma
            # s_i, s_o nuc and ms_i ms_o DM are unique by groupby
            key = (sigma, m_sigma)
            out = data.get(key, S(0))
            data[key] = out + row["val"] * pwd_fact.subs(
                {**row, "sigma": sigma, "m_sigma": m_sigma}
            ).replace(CG, get_cg)

    # Run integrations
    for key, val in data.items():
        data[key] = integrate(subs_all(val), ("Phi", 0, "2*pi"))

    out = Series(data, name="val")
    out.index.names = ("sigma", "m_sigma")
    return out


groups = ["ms_o_dm", "ms_i_dm", "s_o_nuc", "s_i_nuc"]
# run integrations
res = spin_df.reset_index().groupby(groups, as_index=True).agg(op_rank_project)

index_cols = ["sigma", "m_sigma", "ms_o_dm", "ms_i_dm", "s_o_nuc", "s_i_nuc"]
# Drop results which are zero
non_zero_res = DataFrame(res[res != 0]).reset_index().set_index(index_cols).sort_index()
print("Found", non_zero_res.size, "non-zero entries")
non_zero_res.head()

Let's cross check results

In [ ]:
alpha = S("q_3 + 2 * p_i * x_i - 2 * p_o * x_o")
beta1 = S("exp(I*phi) * p_i * sqrt(1 - x_i**2) - p_o * sqrt(1 - x_o**2)")
beta2 = S("exp(-I*phi) * p_i * sqrt(1 - x_i**2) - p_o * sqrt(1 - x_o**2)")
omega = S(
    "4*p_i**2 *(1-x_i**2) + 4 * p_o**2 * (1-x_o**2) - 8*p_i * p_o * cos(phi) * sqrt(1-x_i**2)*sqrt(1-x_o**2)"
).rewrite("exp")
alpha, beta1, beta2

Define the "basis" for comparison

In [ ]:
quotients = {
    S("a**2"): alpha ** 2,
    S("a*b_1"): alpha * beta1,
    S("a*b_2"): alpha * beta2,
    S("b_1**2"): beta1 ** 2,
    S("b_2**2"): beta2 ** 2,
    S("e"): omega,
}

In [ ]:
def decompose(ee):
    """Tries to identify which terms present in quotients describe the input."""
    fact = None
    mat = None
    for k, q in quotients.items():

        try:
            fact = exquo(ee, q)
            mat = k
            break
        except (ExactQuotientFailed, ComputationFailed):
            pass

    out = Series([fact, mat], index=["fact", "mat"])
    return out


decomposition = non_zero_res.val.apply(decompose)
decomposition.head()

Read in Sachin's legacy results for comparison

In [ ]:
legacy = (
    read_csv("data/input-op-32.csv")
    .rename(
        columns={
            "m_chi_p": "ms_o_dm",
            "m_chi": "ms_i_dm",
            "s_p": "s_o_nuc",
            "s": "s_i_nuc",
        }
    )
    .drop(columns=["O", "m_chi_x2", "m_chi_p_x2"])
)
legacy["ms_o_dm"] = legacy["ms_o_dm"].apply(S)
legacy["ms_i_dm"] = legacy["ms_i_dm"].apply(S)
legacy = (
    legacy.set_index(decomposition.index.names)
    .sort_index()
    .applymap(lambda el: S(el.replace("Sqrt", "sqrt")))
)
legacy.head()

If both frames would differ, this would raise an error (so nothing happening means they agree)

In [ ]:
assert_frame_equal(legacy, decomposition)

## Run export used for angular integration

The factor used now is slightly different and instead of summing over $m_s$, we now multiply by different $m_\lambda$.
Generally, the allowed $|m_\lambda| \leq \lambda \leq 2l_\max$ can be quite many, however, because the expression has finite powers of $\exp(i \Phi)$, only a limited number of terms have non-zero results.

In [ ]:
pwd_fact_lambda = S("exp(-I*(m_lambda)*Phi)")
pwd_fact_lambda

In [ ]:
def op_rank_project_lambda(tmp):
    data = dict()
    # sum over ms nuc
    for row in tmp.to_dict("records"):
        # Save results for unique ms DM, s nuc m_lambda
        for m_lambda in range(-2, 2 + 1):
            out = data.get(m_lambda, S(0))
            data[m_lambda] = out + row["val"] * pwd_fact_lambda.subs(
                {**row, "m_lambda": m_lambda}
            )

    # Run angular integrations
    for key, val in data.items():
        data[key] = integrate(subs_all(val), ("Phi", 0, "2*pi"))

    out = Series(data, name="val")
    out.index.name = m_lambda
    return out


groups = ["ms_o_dm", "ms_i_dm", "s_o_nuc", "s_i_nuc", "ms_o_nuc", "ms_i_nuc"]
res_lambda = (
    spin_df.reset_index().groupby(groups, as_index=True).agg(op_rank_project_lambda)
).stack()
res_lambda = DataFrame(res_lambda[res_lambda != 0].sort_index()).rename(
    columns={0: "val"}
)
print("Found", res_lambda.size, "non-zero channels")
res_lambda.index.names = [
    "ms_o_dm",
    "ms_i_dm",
    "s_o_nuc",
    "s_i_nuc",
    "ms_o_nuc",
    "ms_i_nuc",
    "m_lambda",
]
res_lambda.head()